In [ ]:
# Step 1: Install required libraries
!pip install -q roboflow
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt

# Step 2: Import required libraries
import os
from roboflow import Roboflow

# Step 3: Initialize Roboflow with your API key and download the dataset
API_KEY = "BXUACmZ8fN9hjFQwvCKd"
PROJECT_NAME = "construction-site-safety"
VERSION_NUMBER = 30

rf = Roboflow(api_key=API_KEY)
project = rf.workspace("roboflow-universe-projects").project(PROJECT_NAME)
version = project.version(VERSION_NUMBER)
dataset = version.download("yolov5")  # Download dataset in YOLOv5 format

# Step 4: Change directory to YOLOv5
os.chdir("yolov5")

# Step 5: Update data.yaml dynamically based on the dataset
def find_labels_dir(dataset_location):
    """
    Finds the directory containing label files within the dataset location.
    Assumes that labels are organized in the standard YOLO format.
    """
    # Define potential paths for the labels directory
    possible_dirs = [
        os.path.join(dataset_location, "train", "labels"),
        os.path.join(dataset_location, "labels", "train"),
        os.path.join(dataset_location, "labels")
    ]

    # Check for the existence of the directory
    for path in possible_dirs:
        if os.path.exists(path):
            print(f"Labels directory found: {path}")
            return path

    raise ValueError(f"Labels directory not found in expected locations: {possible_dirs}")

def update_data_yaml(dataset_location):
    """
    Updates the `data.yaml` file dynamically based on the dataset structure.
    """
    labels_dir = find_labels_dir(dataset_location)  # Locate labels directory

    # Find all unique label IDs
    all_labels = set()
    for label_file in os.listdir(labels_dir):
        if label_file.endswith(".txt"):
            with open(os.path.join(labels_dir, label_file), "r") as f:
                labels = [line.split()[0] for line in f.readlines()]
                all_labels.update(labels)

    # Create class names dynamically
    labels_list = sorted([int(label) for label in all_labels])
    nc = len(labels_list)
    names = [f'class_{i}' for i in range(nc)]

    # Create data.yaml
    data_yaml_content = f"""
train: {os.path.join(dataset_location, 'train', 'images')}
val: {os.path.join(dataset_location, 'valid', 'images')}

nc: {nc}
names: {names}
"""
    yaml_file = "data.yaml"
    with open(yaml_file, "w") as f:
        f.write(data_yaml_content)

    print(f"data.yaml updated with {nc} classes and saved to {yaml_file}")

# Use the updated function
update_data_yaml(dataset.location)

# Step 6: Train the YOLOv5 model
def train_model():
    try:
        subprocess.run([
            "python", "train.py",
            "--img", "640",  # Image size
            "--batch", "16",  # Batch size
            "--epochs", "5",  # Number of epochs (can be adjusted)
            "--data", "data.yaml",  # Path to dataset YAML
            "--weights", "yolov5n.pt",  # Pretrained weights (adjust if needed)
            "--cache"  # Cache images for faster training
        ], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Training failed: {e}")

train_model()

# Step 7: Get the `best.pt` path dynamically
def get_best_model_path():
    try:
        runs_dir = os.path.join("runs", "train")
        latest_run_dir = max([os.path.join(runs_dir, d) for d in os.listdir(runs_dir)], key=os.path.getmtime)
        best_model_path = os.path.join(latest_run_dir, "weights", "best.pt")
        print(f"Best model path: {best_model_path}")
        return best_model_path
    except Exception as e:
        print(f"Error finding best.pt: {e}")
        return None

best_model_path = get_best_model_path()

# Step 8: Validate the trained model
def validate_model(best_model_path):
    if not best_model_path or not os.path.exists(best_model_path):
        print("Best model not found. Skipping validation.")
        return
    try:
        subprocess.run([
            "python", "val.py",
            "--weights", best_model_path,  # Path to best model
            "--data", "data.yaml",  # Path to dataset YAML
            "--img", "640"  # Image size
        ], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Validation failed: {e}")

validate_model(best_model_path)

# Step 9: Perform inference on test images
def run_inference(best_model_path):
    if not best_model_path or not os.path.exists(best_model_path):
        print("Best model not found. Skipping inference.")
        return
    try:
        test_images_path = os.path.join(dataset.location, "test", "images")
        subprocess.run([
            "python", "detect.py",
            "--weights", best_model_path,  # Path to best model
            "--img", "640",  # Image size
            "--conf", "0.4",  # Confidence threshold
            "--source", test_images_path  # Path to test images
        ], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Inference failed: {e}")

run_inference(best_model_path)

# Step 10: Export the model to ONNX format
def export_model(best_model_path):
    if not best_model_path or not os.path.exists(best_model_path):
        print("Best model not found. Skipping export.")
        return
    try:
        subprocess.run([
            "python", "export.py",
            "--weights", best_model_path,  # Path to best model
            "--include", "onnx"  # Export as ONNX
        ], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Export failed: {e}")

export_model(best_model_path)

print("YOLOv5 workflow completed successfully!")


Cloning into 'yolov5'...
remote: Enumerating objects: 17265, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 17265 (delta 107), reused 60 (delta 60), pack-reused 17079 (from 2)
Receiving objects: 100% (17265/17265), 15.92 MiB | 16.91 MiB/s, done.
Resolving deltas: 100% (11808/11808), done.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Construction-Site-Safety-30 in yolov5pytorch:: 100%|██████████| 1446/1446 [00:00<00:00, 3674.37it/s]


Labels directory found: /content/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5/Construction-Site-Safety-30/train/labels
data.yaml updated with 25 classes and saved to data.yaml
Best model path: runs/train/exp/weights/best.pt
YOLOv5 workflow completed successfully!


In [ ]:
!python train.py --img 640 --batch 16 --epochs 5 --data data.yaml --weights yolov5n.pt --cache


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-01-26 01:16:57.650929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-26 01:16:57.671610: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-26 01:16:57.678146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: shabbirahmedhasan7 (shabbirahmedhasan7-jain-university). Use `wandb login --relogin` to force relogin
train: weights=yolov5n.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scra

In [ ]:
from flask import Flask, render_template, request, jsonify
from flask_socketio import SocketIO, emit
import subprocess
import os

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app, cors_allowed_origins="*")

# Route for the homepage
@app.route('/')
def index():
    return render_template('index.html')

# Route to start YOLOv5 training
@app.route('/train', methods=['POST'])
def train():
    try:
        # YOLOv5 training command
        command = [
            "python", "train.py",
            "--img", "640",  # Image size
            "--batch", "16",  # Batch size
            "--epochs", "5",  # Number of epochs
            "--data", "data.yaml",  # Path to dataset YAML
            "--weights", "yolov5n.pt",  # Pretrained weights
            "--cache"  # Cache images for faster training
        ]

        # Run the training process
        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        # Stream the output
        for line in iter(process.stdout.readline, ""):
            if line:
                socketio.emit('training_output', {'data': line.strip()})
                socketio.sleep(0.1)

        return jsonify({"message": "Training started successfully!"}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Route to perform inference using YOLOv5
@app.route('/detect', methods=['POST'])
def detect():
    try:
        # YOLOv5 detection command
        input_source = request.json.get('source', 'test/images')  # Default source directory
        command = [
            "python", "detect.py",
            "--weights", "best.pt",  # Path to best model weights
            "--img", "640",  # Image size
            "--conf", "0.4",  # Confidence threshold
            "--source", input_source  # Path to test images or video
        ]

        # Run the detection process
        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        # Stream the output
        for line in iter(process.stdout.readline, ""):
            if line:
                socketio.emit('detection_output', {'data': line.strip()})
                socketio.sleep(0.1)

        return jsonify({"message": "Detection started successfully!"}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# SocketIO event to handle training logs
@socketio.on('connect')
def handle_connect():
    print("Client connected")

@socketio.on('disconnect')
def handle_disconnect():
    print("Client disconnected")

if __name__ == '__main__':
    socketio.run(app, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BXUACmZ8fN9hjFQwvCKd")
project = rf.workspace("roboflow-universe-projects").project("construction-site-safety")
version = project.version(30)
dataset = version.download("yolov5-obb")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Construction-Site-Safety-30 in yolov5-obb:: 100%|██████████| 1446/1446 [00:00<00:00, 3937.21it/s]
